In [1]:
## Import statements
import pyspark
from pyspark import SparkConf, SparkContext,SQLContext
from pyspark.sql import HiveContext
from pyspark.sql.types import *
import pyspark.sql.functions as F
from pyspark import StorageLevel
from pyspark.sql.types import *
from pyspark.sql import Row
import re
from operator import itemgetter
import math
import os
from pyspark.sql.types import BooleanType
import pandas as pd
import pyspark # only run after findspark.init()
from pyspark.sql import SparkSession
import time
from random import randint,seed
#import libnum
import sys
import ast
from random import randint
from pyspark.sql.functions import monotonically_increasing_id
from pyspark.sql.functions import row_number,lit
from pyspark.sql.window import Window
from pyspark.sql.functions import col, row_number
from pyspark.sql.types import StructType,StructField, StringType, IntegerType, ArrayType
from pyspark.sql.functions import *

In [48]:
from IPython.core.display import display, HTML
display(HTML("<style>div.jp-OutputArea-output pre {white-space: pre;}</style>"))

/tmp/ipykernel_1839/3453896304.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [2]:
## Building a spark session
spark = SparkSession.builder \
.enableHiveSupport()\
.getOrCreate()

In [6]:
####################### Reading csv file ##########################
### Selecting the columns in particular order
input_df1 = spark.read.option("delimiter",",").option("header","true")\
                        .csv("data_files/hier_data_9_countries_82.csv") #83

w = Window().orderBy(lit('A'))
input_df1 = input_df1.withColumn("id", row_number().over(w))
input_df1 = input_df1.select('id','country','state','county','city','zip_code','super_zipcode')
input_df1.show(10,False)

+---+-------+----------+-------+----------+--------+-------------+
|id |country|state     |county |city      |zip_code|super_zipcode|
+---+-------+----------+-------+----------+--------+-------------+
|1  |US     |California|Alameda|Fremont   |94      |501          |
|2  |US     |California|Alameda|Fremont   |94      |501          |
|3  |US     |California|Alameda|Fremont   |94      |501          |
|4  |US     |California|Alpine |Kirkwood  |95      |646          |
|5  |US     |Texas     |Bailey |Enochs    |79      |324          |
|6  |US     |Texas     |Bailey |Maple     |79      |344          |
|7  |US     |Texas     |Bailey |Muleshoe  |79      |347          |
|8  |US     |Texas     |Bandera|Tarpley   |78      |883          |
|9  |US     |Texas     |Bandera|Vanderpool|78      |885          |
|10 |US     |New York  |Albany |Alcove    |12      |007          |
+---+-------+----------+-------+----------+--------+-------------+
only showing top 10 rows



In [66]:
input_df1.count()

81

In [7]:
# Removing the spaces between two letters
# For eg if we have New York in state column and it will be changed into NewYork
input_df1 = input_df1.withColumn("state",F.regexp_replace(col("state"), " ", ""))\
            .withColumn("county",F.regexp_replace(col("county"), " ", ""))\
            .withColumn("city",F.regexp_replace(col("city")," ",""))

In [8]:
# Listing out all the columns and saving in the list
total_column_list = input_df1.columns
total_column_list

['id', 'country', 'state', 'county', 'city', 'zip_code', 'super_zipcode']

In [9]:
# Getting the length of the columns list
length = len(total_column_list)
print(length)

7


In [10]:
def binary_search(input_df,low,high):
     # Check base case
    rec_cnt_first = input_df.groupBy(total_column_list[1]).agg(F.count('*').alias('cnt')).select(F.min('cnt')).first()[0]
    print("rec_cnt_first:",rec_cnt_first)
    rec_cnt_last = input_df.groupBy([total_column_list[length-1],total_column_list[length-2]]).agg(F.count('*').alias('cnt')).select(F.min('cnt')).first()[0]
    print("rec_cnt_last:",rec_cnt_last)
    if rec_cnt_first < k:
        print(" No hierarchy has more than {0} records".format(k))
        return 1
    if rec_cnt_last > k:
        print(" Even the last hierarchy has more than {0} records".format(k))
        return -1
    if high >= low:
        print("Entering into this if condition")
        mid = (high + low) // 2
        print("mid:",mid)
        # If element is present at the middle itself
        rec_cnt = input_df.groupBy(total_column_list[1:mid+1]).agg(F.count('*').alias('cnt')).select(F.min('cnt')).first()[0]
        print("rec_cnt:",rec_cnt)
        rec_cnt1 = input_df.groupBy(total_column_list[1:mid+2]).agg(F.count('*').alias('cnt')).select(F.min('cnt')).first()[0]
        print("rec_cnt1:",rec_cnt1)
        rec_cnt2 = input_df.groupBy(total_column_list[1:mid]).agg(F.count('*').alias('cnt')).select(F.min('cnt')).first()[0]
        print("rec_cnt2:",rec_cnt2)
        dicta[mid] = rec_cnt
        print("dicta[mid]:",dicta[mid])
        dicta[mid+1]=rec_cnt1
        dicta[mid-1]=rec_cnt2
        if dicta[mid+1] < k and dicta[mid-1]>k:
            if dicta[mid]>=k:
                #print(dicta)
                #print("IF")
                return mid
            else:
                #print(dicta)
                #print("ELSE")
                return mid-1
 
        # If element is smaller than mid, then it can only
        # be present in left subarray
        elif dicta[mid] > k:
            return binary_search(input_df,mid+1,high)
 
        # Else the element can only be present in right subarray
        else:
            return binary_search(input_df, low,mid-1)

    else:
        # Element is not present in the array
        return -1
    

In [11]:
dicta={}
k = 6
col_index = binary_search(input_df1,0,length)
print(col_index)

rec_cnt_first: 1
rec_cnt_last: 1
 No hierarchy has more than 6 records
1


In [12]:
## Bringing back all the columns using column index
column_list =  total_column_list[col_index:]
column_list

['country', 'state', 'county', 'city', 'zip_code', 'super_zipcode']

In [13]:
input_df = input_df1.select([c for c in input_df1.columns if c in column_list])

In [15]:
input_df1.createOrReplaceTempView("temp_main_table")

spark.sql(""" DROP TABLE IF EXISTS main_table """)
spark.sql(""" CREATE TABLE IF NOT EXISTS main_table AS SELECT * FROM temp_main_table """)
spark.sql(""" DROP TABLE IF EXISTS temp_main_table """)


DataFrame[]

In [50]:
spark.sql(" select * from main_table").show(10,False)

+---+-------+----------+-------+----------+--------+-------------+
|id |country|state     |county |city      |zip_code|super_zipcode|
+---+-------+----------+-------+----------+--------+-------------+
|1  |US     |California|Alameda|Fremont   |94      |501          |
|2  |US     |California|Alameda|Fremont   |94      |501          |
|3  |US     |California|Alameda|Fremont   |94      |501          |
|4  |US     |California|Alpine |Kirkwood  |95      |646          |
|5  |US     |Texas     |Bailey |Enochs    |79      |324          |
|6  |US     |Texas     |Bailey |Maple     |79      |344          |
|7  |US     |Texas     |Bailey |Muleshoe  |79      |347          |
|8  |US     |Texas     |Bandera|Tarpley   |78      |883          |
|9  |US     |Texas     |Bandera|Vanderpool|78      |885          |
|10 |US     |NewYork   |Albany |Alcove    |12      |007          |
+---+-------+----------+-------+----------+--------+-------------+
only showing top 10 rows



In [16]:
input_df1.createOrReplaceTempView ("temp_input_table")

spark.sql(""" DROP TABLE IF EXISTS input_table """)
spark.sql(""" CREATE TABLE IF NOT EXISTS input_table AS SELECT * FROM temp_input_table """)
spark.sql(""" DROP TABLE IF EXISTS temp_input_table """)

DataFrame[]

In [52]:
spark.sql(" select * from input_table").show(10,False)

+---+-------+----------+-------+----------+--------+-------------+
|id |country|state     |county |city      |zip_code|super_zipcode|
+---+-------+----------+-------+----------+--------+-------------+
|1  |US     |California|Alameda|Fremont   |94      |501          |
|2  |US     |California|Alameda|Fremont   |94      |501          |
|3  |US     |California|Alameda|Fremont   |94      |501          |
|4  |US     |California|Alpine |Kirkwood  |95      |646          |
|5  |US     |Texas     |Bailey |Enochs    |79      |324          |
|6  |US     |Texas     |Bailey |Maple     |79      |344          |
|7  |US     |Texas     |Bailey |Muleshoe  |79      |347          |
|8  |US     |Texas     |Bandera|Tarpley   |78      |883          |
|9  |US     |Texas     |Bandera|Vanderpool|78      |885          |
|10 |US     |NewYork   |Albany |Alcove    |12      |007          |
+---+-------+----------+-------+----------+--------+-------------+
only showing top 10 rows



In [17]:
# SELECT 1 as flag,country, INT(count(*)) as country_rec_cnt 
#                     FROM input_table group by country ORDER BY country_rec_cnt 

In [18]:
## Performing GroupBy at higher level and obtaining the counts with the help of aggregation
first_count_df = spark.sql(""" SELECT 1 as flag,{0}, INT(count(*)) as {0}_rec_cnt 
                    FROM input_table group by {0} ORDER BY {0}_rec_cnt """.format(column_list[0]))
first_count_df.createOrReplaceTempView("temp1_{0}_count_table".format(column_list[0]))
#first_count_df.show(20,False)

In [19]:
spark.sql(""" DROP TABLE IF EXISTS {0}_count_table """.format(column_list[0]))

spark.sql(""" CREATE TABLE IF NOT EXISTS {0}_count_table AS SELECT * FROM temp1_{0}_count_table """.format(column_list[0]))
spark.sql(""" DROP TABLE IF EXISTS temp1_{0}_count_table""".format(column_list[0]))
spark.sql(""" select * from {0}_count_table """.format(column_list[0])).show(20,False)


+----+-------+---------------+
|flag|country|country_rec_cnt|
+----+-------+---------------+
|1   |FR     |1              |
|1   |CH     |1              |
|1   |DK     |1              |
|1   |AU     |2              |
|1   |BE     |2              |
|1   |ES     |4              |
|1   |JP     |10             |
|1   |US     |23             |
|1   |IN     |37             |
+----+-------+---------------+



In [20]:
# query1: SELECT *, collect_list(country_rec_cnt) 
#     over(PARTITION BY flag ORDER BY monotonically_increasing_id()
#     ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS value  FROM country_count_table
    
# query2: SELECT *,AGGREGATE(value, 0, (acc, el) -> IF(acc < 6, acc + el, el)) as cumsum FROM table1
    
# query3: SELECT *,lag(cumsum,1) over (order by country_rec_cnt) as lag_cumsum FROM table2
    
# query4: SELECT * , sum(CASE WHEN cumsum >=6 AND lag_cumsum >=6 THEN 1  WHEN cumsum <6 AND lag_cumsum >=6 THEN 1 ELSE 0 END) OVER (ORDER BY value) as grp1 FROM table3

# query5: SELECT *,max(cumsum) over(partition by grp1) as maxi FROM table4
    
# query6: SELECT *,(CASE WHEN maxi < 6 THEN grp1-1 ELSE grp1 END) as grp  FROM table_41 
    
# query7: SELECT grp,concat_ws('+',sort_array(collect_list(country))) as country_list FROM table_42
#     group by grp
    
# query8: SELECT  b.country_list,a.*  FROM table_42  a JOIN table5 b ON 
#     a.grp = b.grp 
    
# query9: SELECT collect_set(country) FROM table6 where country = country_list and cumsum >7.199999999999999 
    

In [56]:
def check_k_value(count_table,rec_cnt_field,agg_field,name_field,counter):
    print(f"count_table={count_table}")
    print(f"rec_cnt_field={rec_cnt_field}")
    print(f"agg_field={agg_field}")
    print(f"name_field={name_field}")
    print(f"counter={counter}")
    
    df1 = spark.sql("""SELECT *, collect_list({0}) 
    over(PARTITION BY flag ORDER BY monotonically_increasing_id()
    ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS value  FROM {1}""".format(rec_cnt_field,count_table))
    df1.registerTempTable("table1")
    print("df1")
    df1.show(20,False)
    
    expr1 = "AGGREGATE(value, 0, (acc, el) -> IF(acc < {0}, acc + el, el))".format(k)
    df2 = spark.sql(""" SELECT *,{0} as cumsum FROM table1""".format(expr1))
    df2.registerTempTable("table2")
    print("df2")
    df2.show(20,False)
    
    df3 = spark.sql(""" SELECT *,lag(cumsum,1) over (order by {0}) as lag_cumsum FROM table2""".format(rec_cnt_field))
    df3.registerTempTable("table3")
    print("df3")
    df3.show(20,False)
    
    df4 = spark.sql(""" SELECT * , sum(CASE WHEN cumsum >={0} AND lag_cumsum >={0} THEN 1  WHEN cumsum <{0} AND lag_cumsum >={0} THEN 1 ELSE 0 END) OVER (ORDER BY value) as grp1 FROM table3"""\
                   .format(k))
    df4.registerTempTable("table4")
    print("df4")
    df4.show(20,False)
    
    df_41 = spark.sql("""SELECT *,max(cumsum) over(partition by grp1) as maxi FROM table4  """)
    df_41.registerTempTable("table_41")
    print("df_41")
    df_41.show(20,False)
    
    df_42 = spark.sql("""SELECT *,(CASE WHEN maxi < {0} THEN grp1-1 ELSE grp1 END) as grp  FROM table_41 """.format(k))
    df_42.registerTempTable('table_42')
    print("df_42")
    df_42.show(20,False)
    
    df5 = spark.sql(""" SELECT grp,concat_ws('+',sort_array(collect_list({0}))) as {0}_list FROM table_42
    group by grp""".format(agg_field))
    print("\n GROUPS concatenated \n -----------------------------")
    df5.registerTempTable("table5")
    print("df5")
    df5.show(10,False)

    print("\n Final result \n -----------------------------")
    df6 = spark.sql(""" SELECT  b.{0}_list,a.*  FROM table_42  a JOIN table5 b ON 
    a.grp = b.grp """.format(agg_field))
    print("df6")
    df6.show(10,False)
    df6.registerTempTable("table6")
    
    print("\n Join/Merge back the result \n -----------------------------")
    temp_list= spark.sql(""" SELECT collect_set({1}) FROM table6 where {1} = {1}_list and cumsum >{0} """\
                                            .format(float(1.2*k),agg_field)).collect()[0][0]
    
    print("Function returns the un-aggregated list:",temp_list)
    
    if counter == 0:
        print("In if condition counter value:",counter)
        spark.sql(""" DROP TABLE IF EXISTS {0}_aggregated_table""".format(agg_field))
        spark.sql("""CREATE TABLE IF NOT EXISTS {0}_aggregated_table AS SELECT * FROM table6""".format(agg_field))
        spark.sql("""SELECT * FROM {0}_aggregated_table""".format(agg_field)).show(20,False)
        
        #print("{0},{1}:IF".format(agg_field,counter))
    else:
        print("In else condition counter value:",counter)
        #print("{0},{1}:IF".format(agg_field,counter))
        
        spark.sql(""" DROP TABLE IF EXISTS temp_{0}_aggregated_table""".format(agg_field))
        spark.sql(""" CREATE TABLE IF NOT EXISTS temp_{0}_aggregated_table AS SELECT * FROM table6""".format(agg_field))

        spark.sql(""" INSERT INTO {0}_aggregated_table SELECT * FROM temp_{0}_aggregated_table """.format(agg_field))
        spark.sql(""" SELECT * FROM {0}_aggregated_table LIMIT 20""".format(agg_field)).show(20,False)
        spark.sql(""" DROP TABLE IF EXISTS temp_{0}_aggregated_table""".format(agg_field)) 
        
                
    spark.sql(""" DROP TABLE IF EXISTS table1""")
    spark.sql(""" DROP TABLE IF EXISTS table2""")
    spark.sql(""" DROP TABLE IF EXISTS table3""")
    spark.sql(""" DROP TABLE IF EXISTS table4""")
    spark.sql(""" DROP TABLE IF EXISTS table_41""")
    spark.sql(""" DROP TABLE IF EXISTS table_42""")
    spark.sql(""" DROP TABLE IF EXISTS table5""")
    spark.sql(""" DROP TABLE IF EXISTS table6""")
              
    return temp_list


In [57]:
unaggregated_dict={}
for i in range(1,len(column_list)):
    unaggregated_dict[column_list[i]] = []

In [58]:
print(unaggregated_dict)

{'state': [], 'county': [], 'city': [], 'zip_code': [], 'super_zipcode': []}


In [59]:
unaggregated_dict[column_list[0]] = check_k_value("{0}_count_table".format(column_list[0]),
                                                  "{0}_rec_cnt".format(column_list[0])
                                                  ,"{0}".format(column_list[0]),
                                                  "{0}".format(column_list[0]),0)


count_table=country_count_table
rec_cnt_field=country_rec_cnt
agg_field=country
name_field=country
counter=0
df1
+----+-------+---------------+------------------------------+
|flag|country|country_rec_cnt|value                         |
+----+-------+---------------+------------------------------+
|1   |FR     |1              |[1]                           |
|1   |CH     |1              |[1, 1]                        |
|1   |DK     |1              |[1, 1, 1]                     |
|1   |AU     |2              |[1, 1, 1, 2]                  |
|1   |BE     |2              |[1, 1, 1, 2, 2]               |
|1   |ES     |4              |[1, 1, 1, 2, 2, 4]            |
|1   |JP     |10             |[1, 1, 1, 2, 2, 4, 10]        |
|1   |US     |23             |[1, 1, 1, 2, 2, 4, 10, 23]    |
|1   |IN     |37             |[1, 1, 1, 2, 2, 4, 10, 23, 37]|
+----+-------+---------------+------------------------------+

df2
+----+-------+---------------+------------------------------+------+
|flag|c

In [26]:
print(unaggregated_dict[column_list[0]])

['US', 'IN']


In [76]:
df6.printSchema()

NameError: name 'df6' is not defined

In [27]:
# find countries which need to be drilled down into
counter = 0
counter1 = 0
counter2 = 0
counter3 = 0
counter4 = 0
counter5 = 0
# For  un aggregated countries whose record count is 1.2 times greater than k, need to drill down to states in the country
unaggregated_dict[column_list[counter+1]] = []

for ele in unaggregated_dict[column_list[0]]:
    counter = 1
    if counter >= len(column_list):
        continue
    counter = 1
    print(" {1}  : {0}".format(ele,unaggregated_dict[column_list[0]]))
    temp_list1 =[]
    
    temp_df = spark.sql(""" SELECT 1 as flag,{1},{2}, INT(count(*)) as {2}_rec_cnt 
                        FROM input_table WHERE {1} = '{0}' group by {1},{2} ORDER BY {2}_rec_cnt """.format(ele,column_list[0],column_list[1]))
    temp_df.registerTempTable("temp_{0}_count_table".format(ele))
    
    spark.sql(""" DROP TABLE IF EXISTS {0}_count_table""".format(ele))    
    spark.sql(""" CREATE TABLE IF NOT EXISTS {0}_count_table AS SELECT * FROM temp_{0}_count_table """.format(ele))
    spark.sql(""" DROP TABLE IF EXISTS temp_{0}_count_table""".format(ele))
    
    temp_list1 = check_k_value("{0}_count_table".format(ele),"{0}_rec_cnt".format(column_list[1]),"{0}".format(column_list[1]),ele,counter1)
    print("temp_list1 in for loop:",temp_list1)
    
    counter1 = counter1+1
    print("counter1:",counter1)

    spark.sql(""" DROP TABLE IF EXISTS {0}_count_table""".format(ele))
    if temp_list1:
        print("Going inside temp list1 and it has values")
        unaggregated_dict[column_list[counter]] = temp_list1
    if not temp_list1: # checking if empty
        print("Here the temp_list1 is empty")
        continue
        
    # find states to be drilled into    
    for state_ele in temp_list1:
        counter = 2
        if counter >= len(column_list):
            continue
        print(" {1}  : {0}".format(state_ele,unaggregated_dict[column_list[1]])) 
        temp_list2 = []
        temp_df1 = spark.sql("""SELECT 1 as flag,{2},{3},{4}, INT(count(*)) as {4}_rec_cnt 
                        FROM input_table WHERE {2} = '{0}' AND  {3} = '{1}'
                        group by {2},{3},{4} ORDER BY {4}_rec_cnt """\
                    .format(ele,state_ele,column_list[0],column_list[1],column_list[2]))
        
        temp_df1.registerTempTable("temp_{0}_count_table".format(state_ele))
        
        spark.sql(""" DROP TABLE IF EXISTS {0}_count_table""".format(state_ele))
        spark.sql(""" CREATE TABLE IF NOT EXISTS {0}_count_table AS SELECT * FROM temp_{0}_count_table """.format(state_ele))
        spark.sql(""" DROP TABLE IF EXISTS temp_{0}_count_table""".format(state_ele))
        
        temp_list2=check_k_value("{0}_count_table".format(state_ele),"{0}_rec_cnt".format(column_list[2]),"{0}".format(column_list[2]),state_ele,counter2)
        print("temp_list2:",temp_list2)

        spark.sql(""" DROP TABLE IF EXISTS {0}_count_table""".format(state_ele))
        counter2 = counter2+1
        print("counter2:",counter2)
        if temp_list2:
            print("Going inside temp list2 and it has values")
            unaggregated_dict[column_list[counter]] = temp_list2
        if not temp_list2: # checking if empty
            print("Here the temp_list2 is empty")
            continue
               
        # find counties to be drilled to
        for county_ele in temp_list2:
            counter = 3  
            if counter >= len(column_list):
                continue
            print(" {1}  : {0}".format(county_ele,unaggregated_dict[column_list[2]]))             
            temp_df2 = spark.sql("""SELECT 1 as flag,{3},{4},{5},{6}, INT(count(*)) as {6}_rec_cnt 
                            FROM input_table WHERE {3} = '{0}' AND  {4} = '{1}' AND {5} ='{2}'
                            group by {3},{4},{5},{6} ORDER BY {6}_rec_cnt """\
                        .format(ele,state_ele,county_ele,column_list[0],column_list[1],column_list[2],column_list[3]))
            
            temp_df2.registerTempTable("temp_{0}_count_table".format(county_ele))
            
            spark.sql(""" DROP TABLE IF EXISTS {0}_count_table""".format(county_ele))
            spark.sql(""" CREATE TABLE IF NOT EXISTS {0}_count_table AS SELECT * FROM temp_{0}_count_table """.format(county_ele))
            spark.sql(""" DROP TABLE IF EXISTS temp_{0}_count_table""".format(county_ele))
            
            temp_list3=check_k_value("{0}_count_table".format(county_ele),"{0}_rec_cnt".format(column_list[3]),"{0}".format(column_list[3]),county_ele,counter3)
            print("temp_list3:",temp_list3)
            
            spark.sql(""" DROP TABLE IF EXISTS {0}_count_table""".format(county_ele))
            counter3 = counter3+1
            print("counter3:",counter3)
            if temp_list3:
                print("Going inside temp list3 and it has values")
                unaggregated_dict[column_list[counter]] = temp_list3
            if not temp_list3: # checking if empty
                print("Here the temp_list3 is empty")
                continue
                
            # find cities to be drilled into
            
            for city_ele in temp_list3:
                counter = 4    
                if counter >= len(column_list):
                    continue
                print(" {1}  : {0}".format(county_ele,unaggregated_dict[column_list[3]])) 
                
                temp_df3 = spark.sql("""SELECT 1 as flag,{4},{5},{6},{7},{8}, INT(count(*)) as {8}_rec_cnt 
                FROM input_table WHERE {4} = '{0}' AND  {5} = '{1}' AND {6} ='{2}' AND {7} = '{3}'
                                group by {4},{5},{6},{7},{8} ORDER BY {8}_rec_cnt """\
                            .format(ele,state_ele,county_ele,city_ele,column_list[0],column_list[1],column_list[2],column_list[3],column_list[4]))
                
                temp_df3.registerTempTable("temp_{0}_count_table".format(city_ele))
               
                spark.sql(""" DROP TABLE IF EXISTS {0}_count_table""".format(city_ele))
                spark.sql(""" CREATE TABLE IF NOT EXISTS {0}_count_table AS SELECT * FROM temp_{0}_count_table """.format(city_ele))
                spark.sql(""" DROP TABLE IF EXISTS temp_{0}_count_table""".format(city_ele))

                temp_list4= check_k_value("{0}_count_table".format(city_ele),"{0}_rec_cnt".format(column_list[4]),"{0}".format(column_list[4]),city_ele,counter4)
                print("temp_list4:",temp_list4)
                
                spark.sql(""" DROP TABLE IF EXISTS {0}_count_table""".format(city_ele))
                counter4 = counter4+1
                print("counter4:",counter4)
                
                if temp_list4:
                    print("Going inside temp list4 and it has values")
                    unaggregated_dict[column_list[counter]] = temp_list4
                if not temp_list4: # checking if empty
                    print("Here the temp_list4 is empty")
                    continue    
                    
                for zip_ele in temp_list4:
                    counter = 5    
                    if counter >= len(column_list):
                        continue
                    print(" {1}  : {0}".format(county_ele,unaggregated_dict[column_list[3]])) 

                    temp_df4 = spark.sql("""SELECT 1 as flag,{5},{6},{7},{8},{9},{10}, INT(count(*)) as {10}_rec_cnt 
                    FROM input_table WHERE {5} = '{0}' AND  {6} = '{1}' AND {7} ='{2}' AND {8} = '{3}' AND {9}='{4}'
                                    group by {5},{6},{7},{8},{9},{10} ORDER BY {10}_rec_cnt """\
                                .format(ele,state_ele,county_ele,city_ele,zip_ele,column_list[0],column_list[1],column_list[2],column_list[3],column_list[4],column_list[5]))
                    
                    temp_df4.registerTempTable("temp_{0}_count_table".format(zip_ele))
                    
                    spark.sql(""" DROP TABLE IF EXISTS {0}_count_table""".format(zip_ele))
                    spark.sql(""" CREATE TABLE IF NOT EXISTS {0}_count_table AS SELECT * FROM temp_{0}_count_table """.format(zip_ele))
                    spark.sql(""" DROP TABLE IF EXISTS temp_{0}_count_table""".format(zip_ele))

                    temp_list5= check_k_value("{0}_count_table".format(zip_ele),"{0}_rec_cnt".format(column_list[counter]),"{0}".format(column_list[counter]),zip_ele,counter5)
                    print("temp_list5:",temp_list5)
                    
                    spark.sql(""" DROP TABLE IF EXISTS {0}_count_table""".format(zip_ele))
                    counter5 = counter5+1
                    print("counter5:",counter5)
                    
                    if temp_list5:
                        print("Going inside temp list5 and it has values")
                        unaggregated_dict[column_list[counter]] = temp_list5
                    if not temp_list5: # checking if empty
                        print("Here the temp_list5 is empty")
                        continue



 ['US', 'IN']  : US
df1
+----+-------+----------+-------------+------------------+
|flag|country|state     |state_rec_cnt|value             |
+----+-------+----------+-------------+------------------+
|1   |US     |Virginia  |1            |[1]               |
|1   |US     |Washington|3            |[1, 3]            |
|1   |US     |NewJersey |4            |[1, 3, 4]         |
|1   |US     |California|4            |[1, 3, 4, 4]      |
|1   |US     |Texas     |5            |[1, 3, 4, 4, 5]   |
|1   |US     |NewYork   |6            |[1, 3, 4, 4, 5, 6]|
+----+-------+----------+-------------+------------------+

df2
+----+-------+----------+-------------+------------------+------+
|flag|country|state     |state_rec_cnt|value             |cumsum|
+----+-------+----------+-------------+------------------+------+
|1   |US     |Virginia  |1            |[1]               |1     |
|1   |US     |Washington|3            |[1, 3]            |4     |
|1   |US     |NewJersey |4            |[1, 3, 4]   

In [28]:
# SELECT a.* ,b.cumsum as state_cumsum,grp as state_grp, state_list as state_state_list \
# FROM input_table a LEFT JOIN state_aggregated_table b ON  a.country = b.country AND a.state = b.state 

In [29]:
### Join the aggregated table and master(input) table
## Finally the result will be stored in the table based on the column
def unions_and_joins(master_table,column):
    print("Inside unions and joins")
    q = column_list.index(column)
    concat_table = "{0}_aggregated_table".format(column)
    table_schema = spark.sql("""Show columns in {0}""".format(concat_table)).collect()
    feature_list = [row.col_name for row in table_schema]
    print("feature_list:",feature_list)
    
    temp_query1 = ''
    super_query1 = ''
    for i in range(0,q+1):
        temp_query1 = " a.{0} = b.{0} AND".format(column_list[i])
        super_query1 =  super_query1 + temp_query1
    print("super_query1:",super_query1)

    main_query = """SELECT a.* ,b.cumsum as {0}_cumsum,grp as {0}_grp, {0}_list as {0}_{0}_list FROM \
    {1} a LEFT JOIN {3} b ON {2}""".format(column,master_table,super_query1[:-3],concat_table)
    
    print("main_query of unions and joins:",main_query)
    
    result = spark.sql(main_query)
    print("Unions and Joins result")
    result.show(5,False)
    result.registerTempTable("temp_input_{0}_table".format(column))
    spark.sql(""" DROP TABLE IF EXISTS input_{0}_table""".format(column))

    spark.sql(""" CREATE TABLE IF NOT EXISTS input_{0}_table AS SELECT * FROM temp_input_{0}_table """.format(column))
    spark.sql(""" DROP TABLE IF EXISTS temp_input_{0}_table""".format(column))
    print("UNIONS and JOINS FUNCTION DONE")
    

In [30]:
column = 'country'
mas_tab = 'input_table'
unions_and_joins(mas_tab,column)

Inside unions and joins
feature_list: ['country_list', 'flag', 'country', 'country_rec_cnt', 'value', 'cumsum', 'lag_cumsum', 'grp1', 'maxi', 'grp']
super_query1:  a.country = b.country AND
main_query of unions and joins: SELECT a.* ,b.cumsum as country_cumsum,grp as country_grp, country_list as country_country_list FROM     input_table a LEFT JOIN country_aggregated_table b ON  a.country = b.country 
Unions and Joins result
+---+-------+----------+-------+--------+--------+-------------+--------------+-----------+--------------------+
|id |country|state     |county |city    |zip_code|super_zipcode|country_cumsum|country_grp|country_country_list|
+---+-------+----------+-------+--------+--------+-------------+--------------+-----------+--------------------+
|1  |US     |California|Alameda|Fremont |94      |501          |23            |2          |US                  |
|2  |US     |California|Alameda|Fremont |94      |501          |23            |2          |US                  |
|3  |U

In [31]:
column = 'state'
mas_tab = 'input_table'
unions_and_joins(mas_tab,column)

Inside unions and joins
feature_list: ['state_list', 'flag', 'country', 'state', 'state_rec_cnt', 'value', 'cumsum', 'lag_cumsum', 'grp1', 'maxi', 'grp']
super_query1:  a.country = b.country AND a.state = b.state AND
main_query of unions and joins: SELECT a.* ,b.cumsum as state_cumsum,grp as state_grp, state_list as state_state_list FROM     input_table a LEFT JOIN state_aggregated_table b ON  a.country = b.country AND a.state = b.state 
Unions and Joins result
+---+-------+----------+-------+--------+--------+-------------+------------+---------+----------------+
|id |country|state     |county |city    |zip_code|super_zipcode|state_cumsum|state_grp|state_state_list|
+---+-------+----------+-------+--------+--------+-------------+------------+---------+----------------+
|1  |US     |California|Alameda|Fremont |94      |501          |4           |1        |California+Texas|
|2  |US     |California|Alameda|Fremont |94      |501          |4           |1        |California+Texas|
|3  |US  

In [32]:
# statement:  SELECT country,country_grp,concat_ws('+',sort_array(collect_set(state) over(partition by country_country_list))) as country_state_list,\
#     concat_ws('+',sort_array(collect_set(county) over(partition by country_country_list))) as country_county_list,\
#     concat_ws('+',sort_array(collect_set(city) over(partition by country_country_list))) as country_city_list,\
#     concat_ws('+',sort_array(collect_set(zip_code) over(partition by country_country_list))) as country_zip_code_list,\
#     concat_ws('+',sort_array(collect_set(super_zipcode) over(partition by country_country_list))) as country_super_zipcode_list\
#     FROM input_country_table WHERE country<>country_country_list or
#     country_cumsum <=7.199999999999999
    

In [33]:
# start with column 1
def aggregate_lesser_hierarchy(column,input_table):
    '''collecting the set of remaining hierarchial levels based on the partition column and created the new columns'''
    print("Inside Aggregate lesser hierarchy")
    pos = column_list.index(column)
    temp = ''
    temp1 = ''
    y=''
    x=''
    if pos == 0 :
        for i in column_list[pos+1:]:
            temp = "concat_ws('+',sort_array(collect_set({0}) over(partition by {1}_{1}_list))) as {1}_{0}_list,"\
            .format(i,column)
            temp1 = temp1+temp
    else:   
        for i in column_list[pos+1:]:
            temp = "concat_ws('+',sort_array(collect_set({0}) over(partition by {1}_{1}_list))) as {1}_{0}_list,"\
            .format(i,column)
            temp1 = temp1+temp
    for ele in column_list[0:pos+1]:
        x = "{0},".format(ele)
        y=x+y
        #print("y:",y)
        
    if pos == len(column_list)-1:
        comma= ""
    else:
        comma= ","

    statement = """ SELECT {3},{0}_grp{4}{1} FROM input_{0}_table WHERE {0}<>{0}_{0}_list or
    {0}_cumsum <={2}""".format(column,temp1[:-1],float(1.2*k),y[:-1],comma)
    
    print("statement:",statement)
    
    result = spark.sql(statement)
    #print(result.columns)
    #print(len(result.columns))
    print("result for agg lesser hierarchy")
    result.show(5,False)
    
    result.registerTempTable("temp_all_column_agg_{0}_table".format(column ))
    spark.sql(""" DROP TABLE IF EXISTS  all_column_agg_{0}_table""".format(column))
    spark.sql(""" CREATE TABLE IF NOT EXISTS  all_column_agg_{0}_table AS SELECT * FROM temp_all_column_agg_{0}_table """.format(column))
    spark.sql(""" DROP TABLE IF EXISTS temp_all_column_agg_{0}_table""".format(column))
    print("AGGREGATE LESSER HIERARCHY DONE")
    

In [34]:
column = 'country'
aggregate_lesser_hierarchy(column,"input_{0}_table".format(column))

Inside Aggregate lesser hierarchy
statement:  SELECT country,country_grp,concat_ws('+',sort_array(collect_set(state) over(partition by country_country_list))) as country_state_list,concat_ws('+',sort_array(collect_set(county) over(partition by country_country_list))) as country_county_list,concat_ws('+',sort_array(collect_set(city) over(partition by country_country_list))) as country_city_list,concat_ws('+',sort_array(collect_set(zip_code) over(partition by country_country_list))) as country_zip_code_list,concat_ws('+',sort_array(collect_set(super_zipcode) over(partition by country_country_list))) as country_super_zipcode_list FROM input_country_table WHERE country<>country_country_list or
    country_cumsum <=7.199999999999999
result for agg lesser hierarchy
+-------+-----------+------------------------------------------------------------------------------+------------------------------------------------------------------+-----------------------------------------------------+---------

In [35]:
# statement in post aggregation:  SELECT a.* ,country_state_list,country_county_list,country_city_list,country_zip_code_list,\
#     country_super_zipcode_list FROM input_country_table a LEFT JOIN all_column_agg_country_table b ON  a.country = b.country  


In [36]:
## Joining the above two tables 
def post_aggregation_join(master_table,column):
    print("Inside post aggregation")
    table_name = "all_column_agg_{0}_table".format(column) 
    table_schema = spark.sql("""Show columns in {0}""".format(table_name)).collect()
    feature_list = [ row.col_name for row in table_schema]
    temp_query = ''
    flist = ''
    q=column_list.index(column)
    for feature in feature_list:
        if (re.match("(.*?_)list",feature)):
            flist = "{0},".format(feature)
            temp_query = temp_query + flist
    print("temp_query:",temp_query)
    temp_query1 = ''
    super_query1 = ''
    for i in range(0,q+1):
        temp_query1 = " a.{0} = b.{0} AND".format(column_list[i])
        super_query1 =  super_query1 + temp_query1
        
    if q == len(column_list)-1:
        comma= ""
    else:
        comma= ","
    statement = """ SELECT a.* {3}{0} FROM input_{1}_table a LEFT JOIN all_column_agg_{1}_table b ON {2} """\
    .format(temp_query[:-1],column,super_query1[:-3],comma)
    print("statement in post aggregation:",statement)
   
    result = spark.sql(statement)
    result.registerTempTable("temp_master_{0}_table".format(column))
    print("result of postagg:")
    result.show(5,False)
    spark.sql(""" DROP TABLE IF EXISTS master_{0}_table""".format(column))

    spark.sql(""" CREATE TABLE IF NOT EXISTS master_{0}_table AS SELECT * FROM temp_master_{0}_table """.format(column))
    spark.sql(""" DROP TABLE IF EXISTS temp_master_{0}_table""".format(column))
    print(" POST AGGREGATION JOIN DONE")
    

In [37]:
column = 'country'
post_aggregation_join("input_{0}_table".format(column),column)

Inside post aggregation
temp_query: country_state_list,country_county_list,country_city_list,country_zip_code_list,country_super_zipcode_list,
statement in post aggregation:  SELECT a.* ,country_state_list,country_county_list,country_city_list,country_zip_code_list,country_super_zipcode_list FROM input_country_table a LEFT JOIN all_column_agg_country_table b ON  a.country = b.country  
result of postagg:
+---+-------+----------+-------+--------+--------+-------------+--------------+-----------+--------------------+------------------+-------------------+-----------------+---------------------+--------------------------+
|id |country|state     |county |city    |zip_code|super_zipcode|country_cumsum|country_grp|country_country_list|country_state_list|country_county_list|country_city_list|country_zip_code_list|country_super_zipcode_list|
+---+-------+----------+-------+--------+--------+-------------+--------------+-----------+--------------------+------------------+-------------------+---

In [38]:
for column in column_list:
    print("column:",column)
    posi = column_list.index(column)
    if posi == 0:
        print("Inside if")
        unions_and_joins("input_table",column)
    else:
        print("Inside else")
        unions_and_joins("master_{0}_table".format(column_list[posi-1]),column)
    aggregate_lesser_hierarchy(column,"input_{0}_table".format(column))
    post_aggregation_join("input_{0}_table".format(column),column)
    
    if not unaggregated_dict[column]:
        print("column in unagg dict:",column)
        final_table  = "master_{0}_table".format(column)
        break

column: country
Inside if
Inside unions and joins
feature_list: ['country_list', 'flag', 'country', 'country_rec_cnt', 'value', 'cumsum', 'lag_cumsum', 'grp1', 'maxi', 'grp']
super_query1:  a.country = b.country AND
main_query of unions and joins: SELECT a.* ,b.cumsum as country_cumsum,grp as country_grp, country_list as country_country_list FROM     input_table a LEFT JOIN country_aggregated_table b ON  a.country = b.country 
Unions and Joins result
+---+-------+----------+-------+--------+--------+-------------+--------------+-----------+--------------------+
|id |country|state     |county |city    |zip_code|super_zipcode|country_cumsum|country_grp|country_country_list|
+---+-------+----------+-------+--------+--------+-------------+--------------+-----------+--------------------+
|1  |US     |California|Alameda|Fremont |94      |501          |23            |2          |US                  |
|2  |US     |California|Alameda|Fremont |94      |501          |23            |2          |US

In [39]:
# main_statement:  SELECT super_zipcode,coalesce( country_super_zipcode_list, state_super_zipcode_list, \
#     county_super_zipcode_list) as final_super_zipcode_list,zip_code,\
#     coalesce( country_zip_code_list, state_zip_code_list, county_zip_code_list) as final_zip_code_list,\
#     city,coalesce( country_city_list, state_city_list, county_city_list) as final_city_list,county,\
#     coalesce( country_county_list, state_county_list, county_county_list) as final_county_list,state,\
#     coalesce( country_state_list, state_state_list) as final_state_list,country,\
#     coalesce( country_country_list,country) as final_country_list FROM master_county_table

In [40]:
table_name = final_table
table_schema =spark.sql("""Show columns in {0}""".format(table_name)).collect()
feature_list = [row.col_name for row in table_schema]
super_q1 = ""
super_q2 = ""
for itera in column_list:
    print("itera:",itera)
    tempo = ""
    tempo1= ""
    for feature in feature_list:
        if (re.match(rf"(.*?_){itera}",feature)):
            tempo = " {0},".format(feature)
            print("tempo:",tempo)
            tempo1 =  tempo1 + tempo
            print("tempo1:",tempo1)
    if column_list.index(itera)==0:
        super_q1 = "{1},coalesce({0},{1}) as final_{1}_list".format(tempo[:-1],itera)
        print("super_q1:",super_q1)
    else:
        super_q1 = "{1},coalesce({0}) as final_{1}_list,".format(tempo1[:-1],itera)
        print("super_q1:",super_q1)
    super_q2 = super_q1 + super_q2
    print("super_q2:",super_q2)

main_statement = """ SELECT {0} FROM {1}""".format(super_q2,table_name)
print("main_statement:",main_statement)

final_df = spark.sql(main_statement)
final_df.show(5,False)

itera: country
tempo:  country_country_list,
tempo1:  country_country_list,
super_q1: country,coalesce( country_country_list,country) as final_country_list
super_q2: country,coalesce( country_country_list,country) as final_country_list
itera: state
tempo:  country_state_list,
tempo1:  country_state_list,
tempo:  state_state_list,
tempo1:  country_state_list, state_state_list,
super_q1: state,coalesce( country_state_list, state_state_list) as final_state_list,
super_q2: state,coalesce( country_state_list, state_state_list) as final_state_list,country,coalesce( country_country_list,country) as final_country_list
itera: county
tempo:  country_county_list,
tempo1:  country_county_list,
tempo:  state_county_list,
tempo1:  country_county_list, state_county_list,
tempo:  county_county_list,
tempo1:  country_county_list, state_county_list, county_county_list,
super_q1: county,coalesce( country_county_list, state_county_list, county_county_list) as final_county_list,
super_q2: county,coalesce( 

In [41]:
final_df.registerTempTable("temp_final_agg_table")

spark.sql("""DROP TABLE IF EXISTS  final_agg_table """)

spark.sql(""" CREATE TABLE IF NOT EXISTS  final_agg_table AS SELECT * FROM temp_final_agg_table """)
spark.sql(""" DROP TABLE IF EXISTS temp_final_agg_table """)

DataFrame[]

In [42]:
def join_back_to_main_table():    
    query1 = ''
    super_query1 = ''
    c = [x for x in total_column_list if x not in column_list]
    print(c)
    q2=""
    q3 =""
    for j in c:
        print("j:",j)
        q2 = "a.{0},".format(j)
        q3=q2+q3
        print("q3:",q3)
    for i in column_list:
        query1 = " a.{0} = b.{0} AND".format(i)
        super_query1 =  super_query1 + query1
    print("super_query1:",super_query1)

    main_query = """SELECT DISTINCT {4} b.* FROM \
    {1} a JOIN {3} b ON {2}""".format(column,"main_table",super_query1[:-3],"final_agg_table",q3)
    print(main_query)

    result = spark.sql(main_query)
    result.registerTempTable("final_output_table")
    print("Execution Done")

In [43]:
def join_back_to_main_table():    
    query1 = ''
    super_query1 = ''
    c = [x for x in total_column_list if x not in column_list]
    print(c)
    q2=""
    q3 =""
    for j in c:
        print("j:",j)
        q2 = "a.{0},".format(j)
        q3=q2+q3
        print("q3:",q3)
    for i in column_list:
        query1 = " a.{0} = b.{0} AND".format(i)
        super_query1 =  super_query1 + query1
    print("super_query1:",super_query1)

    main_query = """SELECT {4} b.* FROM \
    {1} a JOIN {3} b ON {2}""".format(column,"main_table",super_query1[:-3],"final_agg_table",q3)
    print(main_query)

    result = spark.sql(main_query)
    result.registerTempTable("final_output_table")
    print("Execution Done")

In [44]:
join_back_to_main_table()

['id']
j: id
q3: a.id,
super_query1:  a.country = b.country AND a.state = b.state AND a.county = b.county AND a.city = b.city AND a.zip_code = b.zip_code AND a.super_zipcode = b.super_zipcode AND
SELECT a.id, b.* FROM     main_table a JOIN final_agg_table b ON  a.country = b.country AND a.state = b.state AND a.county = b.county AND a.city = b.city AND a.zip_code = b.zip_code AND a.super_zipcode = b.super_zipcode 
Execution Done


In [45]:
spark.sql(""" SELECT DISTINCT * FROM final_output_table LIMIT 10""").show(10,False)

+---+-------------+------------------------------------------------------------+--------+--------------------+----------------+--------------------------------------------------------------------------------------------------------------+-------------+------------------------------------------------------------------+-------------+------------------------------------------------------------------------------+-------+------------------+
|id |super_zipcode|final_super_zipcode_list                                    |zip_code|final_zip_code_list |city            |final_city_list                                                                                               |county       |final_county_list                                                 |state        |final_state_list                                                              |country|final_country_list|
+---+-------------+------------------------------------------------------------+--------+--------------------+----------

In [49]:
spark.sql(""" SELECT DISTINCT * FROM final_output_table LIMIT 10""").printSchema()

root
 |-- id: integer (nullable = true)
 |-- super_zipcode: string (nullable = true)
 |-- final_super_zipcode_list: string (nullable = true)
 |-- zip_code: string (nullable = true)
 |-- final_zip_code_list: string (nullable = true)
 |-- city: string (nullable = true)
 |-- final_city_list: string (nullable = true)
 |-- county: string (nullable = true)
 |-- final_county_list: string (nullable = true)
 |-- state: string (nullable = true)
 |-- final_state_list: string (nullable = true)
 |-- country: string (nullable = true)
 |-- final_country_list: string (nullable = true)



In [60]:
spark.sql(""" SELECT id,
final_super_zipcode_list, 
final_zip_code_list,
final_city_list,
final_county_list,
final_state_list,
final_country_list
FROM final_output_table
""").show(100,False)

+---+---------------------------+-------------------+---------------------------------------------------------+-----------------------------+----------------+------------------+
|id |final_super_zipcode_list   |final_zip_code_list|final_city_list                                          |final_county_list            |final_state_list|final_country_list|
+---+---------------------------+-------------------+---------------------------------------------------------+-----------------------------+----------------+------------------+
|3  |324+344+347+501+646+883+885|78+79+94+95        |Enochs+Fremont+Kirkwood+Maple+Muleshoe+Tarpley+Vanderpool|Alameda+Alpine+Bailey+Bandera|California+Texas|US                |
|2  |324+344+347+501+646+883+885|78+79+94+95        |Enochs+Fremont+Kirkwood+Maple+Muleshoe+Tarpley+Vanderpool|Alameda+Alpine+Bailey+Bandera|California+Texas|US                |
|1  |324+344+347+501+646+883+885|78+79+94+95        |Enochs+Fremont+Kirkwood+Maple+Muleshoe+Tarpley+Vanderpool

In [75]:
spark.sql(""" SELECT final_super_zipcode_list,  count(distinct(id)) FROM final_output_table group by final_super_zipcode_list""").show(100,False)

+------------------------------------------------------------+------------------+
|final_super_zipcode_list                                    |count(DISTINCT id)|
+------------------------------------------------------------+------------------+
|324+344+347+501+646+883+885                                 |9                 |
|00+000+10+20+40+89                                          |7                 |
|002+105+409+428+444                                         |7                 |
|01+03+105+120+13+169+20+601                                 |8                 |
|001                                                         |7                 |
|007+009+469+472+737+774                                     |6                 |
|0001+0005+0006+0007+001+002+003+004+0401+0402+1401+1402+1403|14                |
|001+120+701+771                                             |8                 |
|001+016+019+122+124+227                                     |7                 |
|001+002        

In [74]:
spark.sql(""" SELECT final_super_zipcode_list,final_state_list,  count(distinct(id))
FROM final_output_table group by final_super_zipcode_list , final_state_list""").show(100,False)

+------------------------------------------------------------+------------------------------------------------------------------------------+------------------+
|final_super_zipcode_list                                    |final_state_list                                                              |count(DISTINCT id)|
+------------------------------------------------------------+------------------------------------------------------------------------------+------------------+
|00+000+10+20+40+89                                          |AusCapitalTerritory+KantonAargau+Midjylland+NewSouthWales+Vlaanderen+lleFrance|7                 |
|001+120+701+771                                             |UttarPradesh                                                                  |8                 |
|001+002                                                     |Telangana                                                                     |8                 |
|0001+0005+0006+0007+001+002+003+0

In [73]:
spark.sql(""" SELECT final_city_list,final_state_list,  count(distinct(id))
FROM final_output_table group by final_city_list , final_state_list""").show(100,False)

+--------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------+------------------+
|final_city_list                                                                                               |final_state_list                                                              |count(DISTINCT id)|
+--------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------+------------------+
|Fortward+KakinadaH.O+KurupamMarket+VijayawadaH.O+VisakhapatnamH.O+VisakhapatnamPort+Wnchipet                  |AndhraPradesh                                                                 |7                 |
|Aarau+AusNatUniv+Endealve+Grosvenor+Kermt+Lillo+Paris                                                         |AusCapitalTerritory+KantonAargau+Midjylland+

In [72]:
spark.sql(""" SELECT final_super_zipcode_list,final_zip_code_list, 
final_city_list, final_county_list, final_state_list,final_country_list, count(distinct(id)) 
FROM final_output_table 
group by final_super_zipcode_list,final_zip_code_list, 
final_city_list, final_county_list, final_state_list,final_country_list""").show(100,False)

+------------------------------------------------------------+--------------------+--------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------+------------------------------------------------------------------------------+------------------+------------------+
|final_super_zipcode_list                                    |final_zip_code_list |final_city_list                                                                                               |final_county_list                                                 |final_state_list                                                              |final_country_list|count(DISTINCT id)|
+------------------------------------------------------------+--------------------+--------------------------------------------------------------------------------------------------------------+------------------------------------------------

In [71]:
spark.sql(""" SELECT count(distinct(id)) FROM final_output_table""").show(1,False)

+------------------+
|count(DISTINCT id)|
+------------------+
|81                |
+------------------+



In [ ]:
# /**
# final_super_zipcode_list, 
# final_zip_code_list,
# final_city_list,
# final_county_list,
# final_state_list,
# final_country_list
# FROM final_output_table
# **/